In [2]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.plotly as py
import plotly
import statsmodels.api as sm
from scipy.stats import skewnorm as skn
from scipy.stats import norm
import statsmodels.api as sm
import plotly.graph_objs as go

C:\Users\Fang\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [3]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict


def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(len(dataframe))]
    )


# Looping through the soup lxml text table format
# and splitting each row as a individual string
# and parsing string to retrieve the date,
# open, and close information.

def yahoo_table_parse(raw_html_table):
    tickers = []
    call_times = []
    implied_7_day = []
    implied_weekly = []
    eps = []
    i = 1
    end_row = 10
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        row_items = individual_row[0].split('</span>')[:3]

        if i == 1:
            i += 1
            continue
        tick = row_items[0].split('data-symbol="')[1].split('"')[0]
        os_check = optionslam_scrape(tick)

        if type(os_check) == str:
            continue
        else:
            tickers.append(tick)
            call_times.append(row_items[0].split('data-symbol="')[1].split('"')[-1].replace('>',''))
            eps.append(row_items[1].split('</td>')[1].split('>')[1])
            implied_7_day.append(os_check['Current 7 Day Implied'].replace('%',''))
            implied_weekly.append(os_check['Implied Move Weekly'].replace('%',''))


    return pd.DataFrame({'Tickers': tickers, 'Call Times': call_times, 'EPS': eps,
                         'Current 7 Day Implied': implied_7_day,
                         'Implied Move Weekly': implied_weekly})


def yahoo_earnings(date):
    # Yahoo Earnings Calendar Check

    today = date.strftime('%Y-%m-%d')
    tables = []
    for i in range(6):
        yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
        res = requests.get(yahoo_url)
        soup = bs(requests.get(yahoo_url).text, "lxml")

        try:
            table = soup.find_all('table')[0]
            tables.append(yahoo_table_parse(table))
        except:
            print('No Table')

    return pd.concat(tables,axis = 0, ignore_index = True)


def close_data(ticker_lst, start_date = dt.datetime(2018, 2, 20)):
    # Define which online source one should use
    data_source = 'yahoo'

    end = dt.datetime.today()

    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    panel_data = datareader.DataReader(ticker_lst, data_source, start_date, end)

    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    return panel_data.ix['Close']

'''
Function for pulling latest SPX, VIX, VVIX, or SKEW data. Input is a string, pulls 
the latest 2 lines of data from yahoo finance for given ticker and returns a 
dataframe of the open and close with the latest date as the first row.
'''
def latest_yahoo(ticker):
    # Using requests to ping yahoo finance to retrieve 
    # historical data table
    site = 'https://finance.yahoo.com/quote/{0}/history?p={0}'.format(ticker)
    
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    end_row = 3
    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        
        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = [item.split('>')[1] for item in [string.split('</span>')[0] for string in individual_row[0].split('<span ')[1:]]]
        
        if i == 1:
            # Skip first row because they are column headers
            i += 1
            continue
        elif i == end_row:
            break
        else:
            # Append necessary items to initialized lists for 
            # dataframe storage
            close = float(row_items[5].replace(',',''))
        i += 1
    
    # Return dataframe of necessary values
    return np.round(np.float(close),2)

# Function for calculating standard dev and price moves in terms of standard dev
# DF[[Adj Close]] Rolling Period --> DF[['Daily Vol','Daily Price Vol','Price Dev','Annual Vol']]
def price_devs(ticker, lookbackwindow, rollingperiod):
    # Define which online source one should use
    data_source = 'yahoo'
    
    end = dt.datetime.today()
    start_date = end - dt.timedelta(days = lookbackwindow)
    
    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    df = datareader.DataReader([ticker], data_source, start_date, end).sort_index()

    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    df = df.ix['Close'].sort_index()
    
    df.columns = ['prices']
    df['prices_delta'] = df.prices - df.prices.shift(1)
    df['log_returns'] = np.log(df.prices) - np.log(df.prices.shift(1))
    df['daily_vol'] = st.rolling_std(df.log_returns, rollingperiod, ddof = 1)
    df['daily_vol_dollar'] = df.daily_vol*df.prices
    df['price_dev'] = df.prices_delta/df.daily_vol_dollar.shift(1)
    df['annual_vol'] = df.daily_vol*np.sqrt(252)
    return df

In [5]:
# Setup app
# server = flask.Flask(__name__)
# server.secret_key = os.environ.get('secret_key', 'secret')
# app = dash.Dash(__name__, server=server, url_base_pathname='/dash/gallery/volatility-surface', csrf_protect=False)
app = dash.Dash()

external_css = ["https://fonts.googleapis.com/css?family=Overpass:300,300i",
                "https://cdn.rawgit.com/plotly/dash-app-stylesheets/dab6f937fd5548cebf4c6dc7e93a10ac438f5efb/dash-technical-charting.css"]

for css in external_css:
    app.css.append_css({"external_url": css})

if 'DYNO' in os.environ:
    app.scripts.append_script({
        'external_url': 'https://cdn.rawgit.com/chriddyp/ca0d8f02a1659981a0ea7f013a378bbd/raw/e79f3f789517deec58f41251f7dbb6bee72c44ab/plotly_ga.js'
    })


# Plot Fields
returncolumns = ['price_dev','log_returns']
plotfields = [dict(label=str(x), value=str(x)) for x in returncolumns]

# Current data table for graphing
# curr_data = np.round(datacollect.curr_table.tail(),2)
# curr_data['Date'] = datacollect.curr_table.tail().index
# curr_data = curr_data.iloc[:,::-1]

#curr_data = yahoo_earnings()

# Make app layout
app.layout = html.Div(
    [# Titles
        html.Div([
            html.Img(
                src="http://fchen.info/wp-content/uploads/2016/10/fclogo2.png",
                className='five columns',
                style={
                    'height': '60',
                    'width': '60',
                    'float': 'left',
                    'text-align': 'center'
                },
            ),
            html.H1(
                'Earnings Checking Tool',
                className='seven columns',
                style={'text-align': 'center'}
            ),
        ],
            className='row'
        ),
        # First Row
        html.Hr(style={'margin': '0', 'margin-bottom': '5'}),
        html.Div([
            html.Div([
                html.Label('Historical Data Checking:'),
                html.Div([
                    html.Label('Lookback Window:'),
                    dcc.Slider(
                        id='lookback-slider',
                        min = 100,
                        max = 10000,
                        value = 500,
                    ),
                ],
                className = 'row',
                style={'padding': '3%'}),
                html.Div([
                    html.Label('Days for Volatility Calculation:'),
                    dcc.Slider(
                        marks={i: '{}'.format(i) for i in range(5,30)},
                        id='voldays-slider',
                        min = 5,
                        max = 30,
                        value = 20,
                    )
                ],
                className = 'row',
                style={'padding': '3%'}),
                
                html.Div([
                    html.Label('Ticker Lookup'),
                    dcc.Input(id='ticker-input', type='text', value='AAPL')
                ],
                className = 'row',
                style={'padding': '3%'}),
                
                html.Div([
                    html.Label('Histogram Field'),
                    dcc.Dropdown(
                            id='hist-dropdown',
                            options=plotfields,
                            value='price_dev',
                    )
                ],
                className = 'row',
                style={'padding': '3%'}),
                
                html.Button(id='submit-ticker', n_clicks=0, children='Ticker Lookup')
            ],
                className='eight columns',
            ),
            html.Div([
                
                html.Div([
                    html.Label('Earnings Date:'),
                    dcc.DatePickerSingle(
                        id='earnings-date',
                        date=dt.datetime.now().date()
                    ),
                    html.Button(id='submit-yahoo', n_clicks=0, children='Update Date')
                ],
                className = 'row',
                style={'padding': '3%'}),
                                
                html.Div([
                    html.Label('Call Time Filter:'),
                    dcc.RadioItems(
                        id='open_close_selector',
                        options=[
                            {'label': 'Before Market Open', 'value': 'Before Market Open'},
                            {'label': 'After Market Close', 'value': 'After Market Close'},
                        ],
                        value = 'Before Market Open',
                        labelStyle={'display': 'inline-block'},
                    ),
                    html.Button(id='submit-filter', n_clicks=0, children='Call Time Update')
                ],
                className = 'row',
                style={'padding': '3%'})
            ],
                className='four columns',
            )
        ],
            className='row',
            style={'margin-bottom': '1%'}
        ),
        
                
        ## Second Row
        html.Div([
            html.Div([
                html.Div([
                    dcc.Graph(id = 'return-distribution', style={'max-height': '450', 'height': '60vh'})
                ],
                    className = 'row'),
                html.Div([
                    dcc.Graph(id = 'return-history', style={'max-height': '450', 'height': '60vh'})
                ],
                    className = 'row'),
                html.Div(id = 'largest-drop', className = 'row'),
                html.Div(id = 'largest-spike', className = 'row')
            ],
                className='eight columns'
            ),
            html.Div(id = 'worst-table', className = 'four columns')
        ],
            className='row',
            style={'margin-bottom': '2%'}
        ),
        
        
        # Temporary hack for live dataframe caching
        # 'hidden' set to 'loaded' triggers next callback
        html.P(
            hidden='',
            id='earnings-data',
            style={'display': 'none'}
        )
    ],
    style={
        'width': '85%',
        'max-width': '1200',
        'margin-left': 'auto',
        'margin-right': 'auto',
        'font-family': 'overpass',
        'background-color': '#FFFFFF',
        'padding': '40',
        'padding-top': '20',
        'padding-bottom': '20',
    },
)

# Cache raw data
# Callback function to load worst return recalculated data
# into global worst_return_data for useage
@app.callback(Output('earnings-data', 'hidden'),
              [Input('submit-yahoo', 'n_clicks')],
              [State('earnings-date', 'date')])
def cache_raw_data(n_clicks, datepick):
    date_string = dt.datetime.strptime(datepick, '%Y-%m-%d')    
    global earnings_table
    earnings_table = yahoo_earnings(date_string)
    
    print('Loaded raw data')

    return 'Earnings loaded'

@app.callback(Output('worst-table', 'children'),
              [Input('submit-filter', 'n_clicks')],
              [State('open_close_selector', 'value')])
def earnings_table_data(n_clicks, calltime):
    output_table = earnings_table.copy()
    output_table['Close'] = np.nan
    for idx, row in earnings_table.iterrows():
        output_table['Close'].loc[idx] = latest_yahoo(row['Tickers'])
        time.sleep(1)
    output_table['Implied Down Price'] = np.round((1 - output_table['Current 7 Day Implied'].apply(pd.to_numeric)/100)*output_table['Close'],2)
    
    return generate_table(output_table[output_table['Call Times'] == calltime])

# Callback function for yahoo latest options data
@app.callback(Output('return-distribution', 'figure'),
              [Input('submit-ticker', 'n_clicks')],
              [State('ticker-input', 'value'),
               State('lookback-slider', 'value'),
               State('voldays-slider', 'value'),
               State('hist-dropdown', 'value')])
def update_histogram(n_clicks, ticker, lookbackwindow, rollingperiod, histfield):
    
    global retdata
    retdata = price_devs(ticker, lookbackwindow, rollingperiod)[histfield].dropna()

    trace1 = go.Histogram(
        x=retdata,
        histnorm='count',
        name='control',
#         xbins=dict(
#             start=-2.0,
#             end=2.0,
#             size=0.001
#         ),
        opacity=0.75
    )

    data = [trace1]

    layout = go.Layout(
        title= ticker + ' Return Distribution',
        xaxis=dict(
            title=histfield
        ),
        yaxis=dict(
            title='Count'
        ),
        bargap=0.2,
        bargroupgap=0.1
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

# Callback function for yahoo latest options data
@app.callback(Output('return-history', 'figure'),
              [Input('submit-ticker', 'n_clicks')],
              [State('ticker-input', 'value')])
def update_bar(n_clicks, ticker):
    
    trace1 = go.Bar(
        x=retdata.index,
        y=retdata.values,
        name=retdata.name
    )

    data = [trace1]
    layout = go.Layout(
        title=ticker + ' Historical Profile',
        xaxis=dict(
            title = 'Date',
            tickfont=dict(
                size=14
            )
        ),
        yaxis=dict(
            title=retdata.name,
            titlefont=dict(
                size=16
            ),
            tickfont=dict(
                size=14
            )
        ),
        bargap=0.15
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

# Callback function for yahoo latest options data
@app.callback(Output('largest-drop', 'children'),
              [Input('submit-ticker', 'n_clicks')],
              [State('ticker-input', 'value')])
def update_large_drop(n_clicks, ticker):
    
    # Finding date of largest drop
    dropstring = '''
    Largest drop for {2} was {0} and occured on {1}
    '''.format(np.round(retdata.iloc[retdata.index.get_loc(retdata.idxmin())], 2),
               retdata.idxmin().strftime('%Y-%m-%d'),
               ticker)   
    
    return dropstring

# Callback function for yahoo latest options data
@app.callback(Output('largest-spike', 'children'),
              [Input('submit-ticker', 'n_clicks')],
              [State('ticker-input', 'value')])
def update_large_spike(n_clicks, ticker):
    
    # Finding date of largest drop
    spikestring = '''
    Largest Spike for {2} was {0} and occured on {1}
    '''.format(np.round(retdata.iloc[retdata.index.get_loc(retdata.idxmax())], 2),
               retdata.idxmax().strftime('%Y-%m-%d'),
               ticker)
    
    return spikestring


if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2018 11:51:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 11:51:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 11:51:30] "GET /_dash-dependencies HTTP/1.1" 200 -
[2018-03-21 11:51:30,304] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_req

No Table
No Table
No Table
No Table


127.0.0.1 - - [21/Mar/2018 12:01:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:01:42] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:01:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:01:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:01:42] "POST /_dash-update-component HTTP/1.1" 200 -


No Table
Loaded raw data


127.0.0.1 - - [21/Mar/2018 12:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:04:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:06:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:06:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:06:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:07:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:07:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:07:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:07:22] "POST /_dash-update-component HTTP/1.1" 200 -
C:\U

No Table
No Table
No Table
No Table


127.0.0.1 - - [21/Mar/2018 12:13:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:13:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:13:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:13:07] "POST /_dash-update-component HTTP/1.1" 200 -


No Table
Loaded raw data


127.0.0.1 - - [21/Mar/2018 12:14:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:14:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:14:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2018 12:14:32] "POST /_dash-update-component HTTP/1.1" 200 -
[2018-03-21 12:15:16,051] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Fang\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\Fang\Anaconda3\li